In [8]:
import requests
from bs4 import BeautifulSoup
import datetime
import time
import re

In [11]:
def title_f(item):
    return '\n'.join(list(i.strip() for i in item.find('span', re.compile("title")).strings))

def lid_f(item):
    return '\n'.join(list(i.strip() for i in item.find('span', re.compile("lid")).strings))

def date_section_f(item):
    s_dt = item.find('span', re.compile('date')).find('span').text.split(', ') # s_dt for section and datetime
    s_dt_type = []
    for i in s_dt:
        if any(map(str.isdigit, i)) and ':' in i:
            s_dt_type.append('t') # time
        elif any(map(str.isdigit, i)) and set(i.split()) & months != set():
            s_dt_type.append('d') # day
        else:
            s_dt_type.append('s') # section
    if len(s_dt) == 2:
        date = datetime.datetime.strptime(datetime.date.today().isoformat() + s_dt[s_dt_type.index('t')], '%Y-%m-%d%H:%M')
    elif len(s_dt) == 3:
        day = s_dt[s_dt_type.index('d')]
        for old, new in [('янв', '01'), ('фев', '02'), ('мар','03')]:
            day = day.replace(old, new)
        date = datetime.datetime.strptime(day + ' 2023 ' + s_dt[s_dt_type.index('t')], '%d %m %Y %H:%M')
    else:
        date = 'unk'
    section = s_dt[s_dt_type.index('s')]
    return date, section

def text_f(soup_item):
    item_text = soup_item.body.find_all('p')
    text = ''
    for p in item_text:
        abstract = p.text.replace('\n', ' ').strip().replace('\xa0', ' ').replace('  ', '') + '\n'
        text += abstract
    return text

def id_f(item):
    return item.get('id')

def news_feed_f(soup):
    news_list = soup.body.find_all('a', 'news-feed__item chrome js-yandex-counter')
    for item in news_list:
        title = title_f(item)
        date, section = date_section_f(item)
        resp_item = requests.get(item.get('href'))
        soup_item = BeautifulSoup(resp_item.content, 'html.parser')
        text = text_f(soup_item)
        id = id_f(item)[12:]
        news_dict[id] = {'title': title, 'date': date, 'section': section, 'text': text}

def issue_f(href, spec_date):
    resp_issue = requests.get(href)
    soup_issue = BeautifulSoup(resp_issue.content, 'html.parser')
    news_list_issue = soup_issue.body.find_all('a', re.compile('^newspaper-page__news'))
    for item in news_list_issue:
        title = lid_f(item)
        resp_item = requests.get(item.get('href'))
        soup_item = BeautifulSoup(resp_item.content, 'html.parser')
        date = datetime.datetime.strptime(soup_item.body.find('span', 'article__header__date').get('content')[:16], '%Y-%m-%dT%H:%M')
        section = soup_item.body.find('div', re.compile('^rbcslider__slide')).get('data-category')
        id = soup_item.body.find('div', re.compile('^rbcslider__slide')).get('data-id')
        text = text_f(soup_item)
        news_dict[id] = {'title': title, 'date': date, 'section': section, 'text': text}
    if date > spec_date:
        prev_issue_href = soup_issue.body.find('a', re.compile('link_prev')).get('href')
        issue_f(prev_issue_href, spec_date)
    else:
        pass

In [ ]:
# news_dict = {}

In [14]:
page = 'https://www.rbc.ru/newspaper/'
months = {'янв', 'фев', 'мар'}
spec_date = datetime.datetime(2022, 12, 31, 23, 59)
   
resp = requests.get(page)
soup = BeautifulSoup(resp.content, 'html.parser')

# лента новостей текущая
news_feed_f(soup)
print('news_feed is parsed, time: ', datetime.datetime.now())

# свежий номер и все предыдущие номера, включая spec_date
issue = soup.body.find('a', 'header__project__menu__link')
issue_href = issue.get('href')
issue_f(issue_href, spec_date)
print('issues are parsed, time: ', datetime.datetime.now())

# лента новостей обновленная
for i in range(4):
    time.sleep(60 * 30)
    resp = requests.get(page)
    soup = BeautifulSoup(resp.content, 'html.parser')
    news_feed_f(soup)
    print('news_feed is parsed, time: ', datetime.datetime.now())


news_feed is parsed, time:  2023-03-03 12:47:49.828629
issues are parsed, time:  2023-03-03 12:48:47.141507
news_feed is parsed, time:  2023-03-03 13:18:54.916429
news_feed is parsed, time:  2023-03-03 13:49:04.007670
news_feed is parsed, time:  2023-03-03 14:19:13.108637
news_feed is parsed, time:  2023-03-03 14:49:22.035875


In [15]:
len(news_dict)

265

In [17]:
import pandas as pd
import os

fieldnames = ['id', 'title', 'date', 'section', 'text']
df = pd.DataFrame.from_dict(news_dict, orient='index')

df.to_csv('lab01.csv', encoding='utf-8-sig')

In [31]:
df_1 = pd.read_csv('lab01.csv')

In [32]:
df_1

,Unnamed: 0,title,date,section,text
0,63fcd0ca9a79479091218d5f,Как устроен индустриальный интернет вещей,2023-02-28 16:43:00,РБК и Ростелеком,Первыми мессенджерами в истории человека были ...
1,currencycbeur,"Курс евро на 1 марта\nEUR ЦБ: 79,51 (-0,11)",2023-02-28 16:08:00,Инвестиции,NaN
2,currencycbusd,"Курс доллара на 1 марта\nUSD ЦБ: 74,89 (-0,54)",2023-02-28 16:08:00,Инвестиции,NaN
3,63ad93b59a794725346b679c,Из чего на самом деле состоят полуфабрикаты,2023-02-28 12:53:00,РБК и Черкизово,NaN
4,63fc655f9a79475c41070cdd,От сигнальных огней до мессенджеров: как разви...,2023-02-28 12:32:00,РБК и Ростелеком,Первыми мессенджерами в истории человека были ...
...,...,...,...,...,...
260,64010c9a9a79478e944069b5,В США задержали двух американцев за передачу Р...,2023-03-02 23:58:00,Политика,Два гражданина США были задержаны в Канзас-Сит...
261,6400f4d39a79478a7e1e70f5,В СДПГ отказались исключить Шрёдера за связи с...,2023-03-02 23:54:00,Политика,Арбитражная комиссия второй инстанции партии С...
262,640103e19a79478cddc83b84,Австрия назвала нормальной проверку США «Райфф...,2023-03-02 23:38:00,Финансы,Министерство финансов Австрии не увидело повод...
263,640102b79a79478c6eca517c,Машина Росгвардии подорвалась на мине в Брянск...,2023-03-02 23:18:00,Политика,В Брянской области автомобиль Росгвардии подор...
